In [1]:
#coding:utf-8
import tensorflow as tf
import numpy as np

import warnings
warnings.filterwarnings('ignore')

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

tfrecord_path = '../data/basic.tfrecord'

/home/ubuntu/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# write
# step1
writer = tf.python_io.TFRecordWriter(tfrecord_path)

# step2
for i in range(2):
    a = 0.68 + i
    b = [10 + i, 11 + i]
    c = np.array([[1, 2, 3], [4, 5, 6]]) + i
    c = c.astype('uint8')
    c_raw = c.tostring()
    # step3
    example = tf.train.Example(features=tf.train.Features(feature={
        'a': tf.train.Feature(float_list=tf.train.FloatList(value=[a])),
        'b': tf.train.Feature(int64_list=tf.train.Int64List(value=b)),
        'c': tf.train.Feature(bytes_list=tf.train.BytesList(value=[c_raw]))
    }))
    # step4
    serialized = example.SerializeToString(example)
    writer.write(serialized)

# step5
writer.close()

In [2]:
# read
# step1
filename_queue = tf.train.string_input_producer([tfrecord_path], num_epochs=None)
# step2
reader = tf.TFRecordReader()
# step3
_, serialized_example = reader.read(filename_queue)
# step4
features = tf.parse_single_example(serialized_example, features={
    'a': tf.FixedLenFeature([], tf.float32),
    'b': tf.FixedLenFeature([2], tf.int64),
    'c': tf.FixedLenFeature([], tf.string)
})
# step5
a_out = features['a']
b_out = features['b']
c = features['c']
c_out = tf.decode_raw(c, tf.uint8)
c_out = tf.reshape(c_out, [2, 3])

init = tf.initialize_all_variables()
sess.run(init)
print('a_out', a_out)
print('b_out', b_out)
print('c_out', c_out)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
a_out Tensor("ParseSingleExample/Squeeze_a:0", shape=(), dtype=float32)
b_out Tensor("ParseSingleExample/Squeeze_b:0", shape=(2,), dtype=int64)
c_out Tensor("Reshape:0", shape=(2, 3), dtype=uint8)


In [4]:
# shuffle batch
a_batch, b_batch, c_batch = tf.train.shuffle_batch([a_out, b_out, c_out], batch_size=3, capacity=200,
                                                   min_after_dequeue=100, num_threads=2)

coord = tf.train.Coordinator()
tf.train.start_queue_runners(sess=sess, coord=coord)
a_val, b_val, c_val = sess.run([a_batch, b_batch, c_batch])
print('a_val', a_val)
print('b_val', b_val)
print('c_val', c_val)

a_val [0.68 0.68 1.68]
b_val [[10 11]
 [10 11]
 [11 12]]
c_val [[[1 2 3]
  [4 5 6]]

 [[1 2 3]
  [4 5 6]]

 [[2 3 4]
  [5 6 7]]]
